## Ring Inflation of the Triangle - Compatibility Test
---

In [1]:
import gurobipy as gp
import numpy as np
from collections import defaultdict

In [2]:
P_ABC = {(0,0,0): 0,
         (0,0,1): 1/3,
         (0,1,0): 1/3,
         (1,0,0): 1/3,
         (0,1,1): 0,
         (1,0,1): 0,
         (1,1,0): 0,
         (1,1,1): 0}

# P_ABC = {(0,0,0): 1/4,
#          (0,0,1): 1/4,
#          (0,1,0): 1/4,
#          (1,0,0): 1/4,
#          (0,1,1): 0,
#          (1,0,1): 0,
#          (1,1,0): 0,
#          (1,1,1): 0}

def get_funcs(P_ABC):
    P_A = np.array([sum([P_ABC[a,b,c] for b,c in np.ndindex(2,2)]) for a in range(2)])
    P_B = np.array([sum([P_ABC[a,b,c] for a,c in np.ndindex(2,2)]) for b in range(2)])
    P_C = np.array([sum([P_ABC[a,b,c] for a,b in np.ndindex(2,2)]) for c in range(2)])

    P_AB = np.array([[sum([P_ABC[a,b,c] for c in range(2)]) for b in range(2)] for a in range(2)])
    P_AC = np.array([[sum([P_ABC[a,b,c] for b in range(2)]) for c in range(2)] for a in range(2)])
    P_BC = np.array([[sum([P_ABC[a,b,c] for a in range(2)]) for c in range(2)] for b in range(2)])

    return P_A, P_B, P_C, P_AB, P_AC, P_BC

P_A, P_B, P_C, P_AB, P_AC, P_BC = get_funcs(P_ABC)

In [3]:
m = gp.Model()
m.params.InfUnbdInfo = 1

Q_A1_A2_B1_B2_C1_C2 = m.addMVar((2,2,2,2,2, 2), name="P", vtype=gp.GRB.CONTINUOUS, lb=0, ub=1)
m.update()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-09
Set parameter InfUnbdInfo to value 1


In [4]:
Q_shape = (2,2,2,2,2,2)
n_col = np.prod(Q_shape)
Q_base = np.eye(64).reshape(Q_shape+(n_col,))

b_vec = []
b_vec_symbolic = []

A_mat = []

for a1, a2, b1, b2 in np.ndindex(2, 2, 2, 2):
    A_mat.append(sum(sum(Q_base[a1,a2,b1,b2,:,:]))) # Q_A1_A2_B1_B2(a1,a2,b1,b2)
    b_vec.append(P_AB[a1,b1]*P_AB[a2,b2])
    b_vec_symbolic.append("*".join(sorted([f"P_AB({a1},{b1})",f"P_AB({a2},{b2})"])))


for b1, b2, c1, c2 in np.ndindex(2, 2, 2, 2):
    A_mat.append(sum(sum(Q_base[:,:,b1,b2,c1,c2]))) # Q_B1_B2_C1_C2(b1,b2,c1,c2)
    b_vec.append(P_BC[b1,c1]*P_BC[b2,c2])
    b_vec_symbolic.append("*".join(sorted([f"P_BC({b1},{c1})",f"P_BC({b2},{c2})"])))


for a1, a2, c1, c2 in np.ndindex(2, 2, 2, 2):
    A_mat.append(sum(sum(Q_base[a1,a2,:,:,c1,c2]))) # Q_A1_A2_C1_C2(a1,a2,c1,c2)
    b_vec.append(P_AC[a1,c2]*P_AC[a2,c1])
    b_vec_symbolic.append("*".join(sorted([f"P_AC({a1},{c2})",f"P_AC({a2},{c1})"])))


for a,b,c in np.ndindex(2, 2, 2):
    # symmetry
    A_mat.append(sum(sum(sum(Q_base[a,:,:,b,c,:])))) # Q_A1_B2_C1(a,b,c)
    b_vec.append(P_A[a]*P_C[c]*P_B[b])
    b_vec_symbolic.append(f"P_A({a})*P_C({c})*P_B({b})")
    
    A_mat.append(sum(sum(sum(Q_base[:,a,b,:,:,c])))) # Q_A2_B1_C2(a,b,c)
    b_vec.append(P_A[a]*P_C[c]*P_B[b])
    b_vec_symbolic.append(f"P_A({a})*P_C({c})*P_B({b})")


m.update()

A = np.asarray(A_mat)
b = np.asarray(b_vec)
y = m.addMVar(shape=len(b), vtype=gp.GRB.CONTINUOUS, name="y", lb=-1, ub=1)
m.setObjective(y @ b, gp.GRB.MINIMIZE)
m.addMConstr(A.T, y, '>=', [0]*n_col)
m.optimize()

print("y⋅b:", m.objVal)

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 64 rows, 128 columns and 320 nonzeros
Model fingerprint: 0x7ad01fd8
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-02, 3e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Presolve removed 13 rows and 85 columns
Presolve time: 0.00s
Presolved: 51 rows, 43 columns, 210 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -4.8518519e+00   2.160000e+02   0.000000e+00      0s
      50   -2.5925926e-01   0.000000e+00   0.000000e+00      0s

Solved in 50 iterations and 0.03 seconds (0.00 work units)
Optimal objective -2.592592593e-01
y⋅b: -0.25925925925925963


In [8]:
A*y

<MLinExpr (64, 64)>
array([[ y[0],  y[1],  y[2], ...,  0.0 y[61],  0.0 y[62],  0.0 y[63]],
       [ 0.0 y[0],  0.0 y[1],  0.0 y[2], ...,  0.0 y[61],  0.0 y[62],
         0.0 y[63]],
       [ 0.0 y[0],  0.0 y[1],  0.0 y[2], ...,  0.0 y[61],  0.0 y[62],
         0.0 y[63]],
       ...,
       [ 0.0 y[0],  0.0 y[1],  0.0 y[2], ...,  0.0 y[61],  y[62],
         0.0 y[63]],
       [ 0.0 y[0],  0.0 y[1],  0.0 y[2], ...,  0.0 y[61],  y[62],  y[63]],
       [ 0.0 y[0],  0.0 y[1],  0.0 y[2], ...,  y[61],  0.0 y[62],  y[63]]])

In [6]:
solution_dict = defaultdict(int)
for name, val in zip(b_vec_symbolic, y.X):
    if val != 0:
        solution_dict[name] += val
for key, val in solution_dict.items():
    if val != 0:
        print(f"{key} * {val}")
# print("Optimal solution:", y.X)

P_AB(0,0)*P_AB(0,0) * 1.0
P_AB(0,0)*P_AB(0,1) * -1.0
P_AB(0,0)*P_AB(1,0) * -2.0
P_AB(0,0)*P_AB(1,1) * 2.0
P_AB(0,1)*P_AB(1,1) * 2.0
P_AB(1,0)*P_AB(1,0) * -1.0
P_AB(1,0)*P_AB(1,1) * 2.0
P_AB(1,1)*P_AB(1,1) * 1.0
P_BC(0,0)*P_BC(0,0) * 1.0
P_BC(0,0)*P_BC(0,1) * -2.0
P_BC(0,1)*P_BC(0,1) * -1.0
P_BC(0,0)*P_BC(1,1) * 2.0
P_BC(0,1)*P_BC(1,1) * 2.0
P_BC(0,0)*P_BC(1,0) * -1.0
P_BC(1,0)*P_BC(1,0) * -1.0
P_BC(1,0)*P_BC(1,1) * 2.0
P_BC(1,1)*P_BC(1,1) * 1.0
P_AC(0,0)*P_AC(0,0) * 1.0
P_AC(0,0)*P_AC(0,1) * -2.0
P_AC(0,1)*P_AC(0,1) * -1.0
P_AC(0,0)*P_AC(1,0) * -2.0
P_AC(0,0)*P_AC(1,1) * 2.0
P_AC(0,1)*P_AC(1,1) * 2.0
P_AC(0,1)*P_AC(1,0) * 1.0
P_AC(1,0)*P_AC(1,0) * -1.0
P_AC(1,0)*P_AC(1,1) * 2.0
P_AC(1,1)*P_AC(1,1) * 1.0
P_A(0)*P_C(0)*P_B(0) * 1.0
P_A(0)*P_C(1)*P_B(0) * 1.0
P_A(0)*P_C(1)*P_B(1) * 2.0
P_A(1)*P_C(0)*P_B(0) * 1.0
P_A(1)*P_C(1)*P_B(0) * 2.0
P_A(1)*P_C(0)*P_B(1) * 2.0
P_A(1)*P_C(1)*P_B(1) * -2.0
